In [ ]:
# All imports
import carla  # The sim library itself
import math
import random  # To pick random spawn point
import cv2  # To work with images from cameras
import numpy as np  # To change image representation - re-shaping
import time # to handle stopping time
from carla import WalkerAIController 
from ultralytics import YOLO
model = YOLO('yolov8n.pt')# Load the YOLOv8 model


In [2]:
# Connect to the sim
client = carla.Client('localhost', 2000)

In [3]:
# Optional to load different towns
client.load_world('Town02')

In [4]:
# Define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [5]:
# Look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')[0]

In [6]:
# Spawn a car in a random location
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp, start_point)

In [7]:
# Move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z + 1  # Move 1m up to avoid being on the floor
spectator.set_transform(start_point)

In [ ]:
# חיבור ל-Traffic Manager
traffic_manager = client.get_trafficmanager(8000) 

In [9]:
# Send the car off on autopilot - this will leave the spectator
vehicle.set_autopilot(True,traffic_manager.get_port())

In [ ]:

# Get pedestrian blueprints
walker_blueprints = world.get_blueprint_library().filter('walker.pedestrian.*')

# Get valid spawn points
spawn_points = world.get_map().get_spawn_points()

# Lists to keep track of pedestrians and their controllers
pedestrians = []
controllers = []
pedestrian_destinations = {}

def update_pedestrian_destinations():
    for pedestrian, controller in zip(pedestrians, controllers):
        current_location = pedestrian.get_location()
        destination = pedestrian_destinations.get(pedestrian.id, None)
        if destination:
            distance = current_location.distance(destination)
            if distance < 1.0:  # Threshold distance to destination (meters)
                new_destination = world.get_random_location_from_navigation()
                if new_destination:
                    controller.go_to_location(new_destination)  # Set a new destination
                    pedestrian_destinations[pedestrian.id] = new_destination  # Update the destination in the dictionary
                    print(f"Pedestrian {pedestrian.id} new destination: {new_destination}")  # Print new destination
def spawn_pedestrian():
    # Choose a random blueprint and spawn point
    walker_bp = random.choice(walker_blueprints)
    spawn_point = random.choice(spawn_points)

    # Attempt to spawn the pedestrian
    pedestrian = world.try_spawn_actor(walker_bp, spawn_point)
    if pedestrian is None:
        return None, None

    # Assign a controller to the pedestrian
    controller_bp = world.get_blueprint_library().find('controller.ai.walker')
    controller = world.try_spawn_actor(controller_bp, carla.Transform(), attach_to=pedestrian)
    if controller is None:
        pedestrian.destroy()
        return None, None

    # Start the controller and set movement parameters
    controller.start()
    controller.go_to_location(world.get_random_location_from_navigation())
    controller.set_max_speed(2.0)  # Set desired speed

    return pedestrian, controller

num_pedestrians = 20  # Number of pedestrians to spawn
world.set_pedestrians_cross_factor(50)
spawn_attempts = 0
max_attempts = num_pedestrians * 2  # Limit to prevent infinite loops

while len(pedestrians) < num_pedestrians and spawn_attempts < max_attempts:
    pedestrian, controller = spawn_pedestrian()
    if pedestrian and controller:
        pedestrians.append(pedestrian)
        controllers.append(controller)
        print(f"Spawned pedestrian {pedestrian.id}")
    else:
        print("Failed to spawn pedestrian, retrying...")
    spawn_attempts += 1
    
print(f"Successfully spawned {len(pedestrians)} pedestrians.")

Spawned pedestrian 664
Spawned pedestrian 666
Spawned pedestrian 668
Spawned pedestrian 670
Spawned pedestrian 672
Spawned pedestrian 674
Spawned pedestrian 676
Spawned pedestrian 678
Spawned pedestrian 680
Spawned pedestrian 682
Spawned pedestrian 684
Spawned pedestrian 686
Spawned pedestrian 688
Spawned pedestrian 690
Spawned pedestrian 692
Spawned pedestrian 694
Spawned pedestrian 696
Spawned pedestrian 698
Spawned pedestrian 700
Spawned pedestrian 702
Successfully spawned 20 pedestrians.


In [ ]:
def get_speed(vehicle):
    velocity = vehicle.get_velocity()
    speed = math.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2) # Vector magnitude of speed
    return speed * 3.6  # Convert to km/h

In [ ]:
# Set camera parameters
CAMERA_POS_Z = 1.6  # Camera height
CAMERA_POS_X = 0.9  # Camera forward position

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# Initialize camera data dictionary for live feed
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image': np.zeros((image_h, image_w, 4))}

is_braking = False  # Whether the vehicle is in braking state
brake_end_time = 0  # Time when braking should end
brake_duration = 5  # Duration of braking in seconds

# Updated camera callback function with YOLO detection
def camera_callback(image, data_dict):
    global is_braking, brake_end_time    
    # Convert CARLA image to a format compatible with OpenCV
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    frame = array[:, :, :3]  # Remove alpha channel
    
    # Detect and track objects
    results = model.track(frame, persist=True)
    
    # Check if any detected object is a pedestrian
    high_confidence_boxes = []
    for result in results:
        for box in result.boxes:
            if box.conf >= 0.6  and (box.cls == 0 or box.cls == 10):   # Only include detections with confidence >= 0.5
                high_confidence_boxes.append(box) 

    # Update results to contain only high-confidence boxes
    results[0].boxes = high_confidence_boxes  # Update results with filtered boxesqq

     # If not already braking and a pedestrian is detected
    if not is_braking and any(box.cls == 0 for box in high_confidence_boxes):
        print("Pedestrian detected! Reducing speed for 5 seconds.")  
        traffic_manager.vehicle_percentage_speed_difference(vehicle,50) # Reduce vehicle speed by 50%
        is_braking = True  # Update braking state
        brake_end_time = time.time() + brake_duration   # Set brake end time
        print(f'Brake End Time: {brake_end_time} | Is Braking: {is_braking}') 

    # Plot the filtered results
    frame_with_boxes = results[0].plot()
    data_dict['image'] = frame_with_boxes

# Attach the updated callback to the camera
camera.listen(lambda image: camera_callback(image, camera_data))


In [ ]:
# Loop to display camera feed
while True:
    current_speed = get_speed(vehicle)
    print(f"--------------------------------Current speed: {current_speed:.2f} km/h-----------------------------")
    update_pedestrian_destinations()
    # Check if braking time has ended
    if is_braking and time.time() >= brake_end_time:
        print("Resuming normal speed.") 
        traffic_manager.vehicle_percentage_speed_difference(vehicle, 0) # Reset speed difference to 0
        current_speed4 = get_speed(vehicle)
        is_braking = False  # Update braking state
        
    world.tick()
    # Display with imshow
    cv2.imshow('All cameras', camera_data['image'])    
    if cv2.waitKey(1) == ord('q'):
        break

# Grab a snapshot from the camera and show in a pop-up window
img = camera_data['image']
cv2.imshow('RGB Camera', img)
cv2.waitKey(0)


0: 384x640 2 benchs, 49.1ms
Speed: 3.0ms preprocess, 49.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 benchs, 41.0ms
Speed: 3.0ms preprocess, 41.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 benchs, 37.0ms
Speed: 3.0ms preprocess, 37.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 benchs, 39.0ms
Speed: 2.0ms preprocess, 39.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 traffic lights, 38.0ms
Speed: 2.0ms preprocess, 38.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 41.0ms
Speed: 4.0ms preprocess, 41.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 38.0ms
Speed: 2.0ms preprocess, 38.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 traffic lights, 1 bench, 64.0ms
Speed: 3.0ms preprocess, 64.0ms inference, 

113

In [ ]:
# Clean up after yourself
cv2.destroyAllWindows()
camera.stop()  # This is the opposite of camera.listen
for actor in world.get_actors().filter('vehicle'):
    actor.destroy()
for sensor in world.get_actors().filter('sensor'):
    sensor.destroy()

# Clean up Padstrians
def clean_up():
    for controller in controllers:
        controller.stop()
        controller.destroy()
    for pedestrian in pedestrians:
        pedestrian.destroy()
    print("Cleaned up all pedestrians and controllers.")

# Register the clean-up function to run on script exit
import atexit
atexit.register(clean_up)

<function __main__.clean_up()>

In [ ]:
# Display the number of spawn points
len(spawn_points)
# Set the spectator's position
start_point = spawn_points[0]
spectator.set_transform(start_point)
spectator.set_transform(carla.Transform(carla.Location(x=0, y=0, z=30))) 

In [ ]:
#Draft
print(model.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

0: 384x640 1 person, 1 traffic light, 96.5ms
Speed: 2.0ms preprocess, 96.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
